In [4]:
#import necessary package and collect initial variables
import math
import json
import numpy as np
import pandas as pd
import prelim_wing_design
import openmdao.api as om
import wing_section_properties
import matplotlib.pyplot as plt
from openaerostruct.utils import plot_wing
from openaerostruct.geometry.utils import generate_mesh
from openaerostruct.geometry.geometry_group import Geometry 
from wing_pressure_loads_calculator import wing_pressure_loads
from openaerostruct. aerodynamics.aero_groups import AeroPoint 
import naca_five_digit_aerofoil_coordinates_calculator as naca_calc
from cruise_conditions_calculator import cruise_conditions_calculator

In [5]:
#define discipline group and call analyses 
class aero(om.Group):
    def initialize(self):
        
        #read all excel constants        
        initial_vars = pd.read_excel('constants.xlsx')
        
        #sort for just the systems constants
        all_aero_vars = np.where(initial_vars['aero'] == True, [initial_vars['variable name'], initial_vars['value'], initial_vars['description']], None)
        
        #create a DataFrame for the systems variables to pull from in individual analyses
        all_aero_vars = pd.DataFrame(all_aero_vars).dropna(axis = 1).transpose()
        
        #pass variables into the correct openMDAO format, passing name, description and value
        i=0
        for variable in all_aero_vars[0]:
            value = all_aero_vars.iloc[i][1]
            description = all_aero_vars.iloc[i][2]
            self.options.declare(variable, default = value, desc = description)
            i+=1
        i=0
        
    def setup(self):
        
        self.add_subsystem("aero_IDO", aero_IDO(), promotes=["*"])
        self.add_subsystem('derivatives_calc',derivatives_calc(), promotes = ['*'])
        
    def configure(self):
        #promote all variables (lazy option, they can be connected individually), will connect automatically if the variable names are the same 
        self.promotes('aero_IDO',any=['*'])
        self.promotes('derivatives_calc',any=['*'])
        

In [8]:
class aero_IDO(om.ExplicitComponent):
    def initialize(self):
        #take constants from group a level above
        self.options.declare('naca_series',prob.model.aero.options['naca_series'])
        self.options.declare('max_b',prob.model.aero.options['max_b'])
        
    def setup(self):
        self.add_input('m')
        
        self.add_output('chord_tip')
        self.add_output('chord_root')
        
        
        
    def compute(self,inputs,outputs):
        #start by redefining your variables using inputs and constants (called self.options)
        mass = inputs['mass']
        naca_series = self.options['naca_series']
        max_allowable_wing_span = self.options['max_b']
        
        #write to file, aero IDO will read them and run and output to a different file
        keyaeronumbers = pd.DataFrame([mass,naca_series,max_allowable_wing_span])
        keyaeronumbers.to_csv("keyAero.dat", index=False)
        #in your IDO use the keyAero file
        
        #since you use openaerostruct, please use different names for the problem and model etc so there are no clashes
        #otherwise your IDO will be messing with the MDO 
        %run aero_ido_vf7.py
        
        #read aero IDO output file 
        #aerodynamic_outputs = read file 
        
        #add outputs to connect to other disciplines
        outputs['chord_tip'] = aerodynamic_outputs["chord tip"]
        outputs['chord_root'] = aerodynamic_outputs["chord root"]
        #carry this on, add to setup area above^


In [ ]:
class derivatives_calc(om.ExplicitComponent):
    def setup(self):
        self.add_output('CL_alpha')
        self.add_output('CD_alpha')
        self.add_output('CM_alpha')
        #etc
    
    def compute(self,inputs, outputs):
        
        #run the derivatives calculator 
        
        outputs['CL_alpha'] = all_derivatives['CL_alpha']
        outputs['CD_alpha'] = all_derivatives['CD_alpha']
        outputs['CM_alpha'] = all_derivatives['CM_alpha']
        #etc